# Runner Notebook - Muhammad Hassaan Sohail
This notebook runs the demo: it seeds the database, starts the MCP server, creates agents, and executes the demo scenarios. Use a Python kernel that has `Flask` and `requests` installed (see `requirements.txt`).

Github Link:

In [1]:
import threading
import time
import subprocess
import requests
import os
import sys

from database_setup import create_db, DB_PATH

def start_mcp_server():
    # Start the Flask MCP server using the same Python interpreter running this notebook
    return subprocess.Popen([sys.executable, 'mcp_server.py'], shell=False)

def wait_for_server(url='http://127.0.0.1:8000/mcp/get_customer/1', timeout=10):
    start = time.time()
    while time.time() - start < timeout:
        try:
            r = requests.get(url)
            return True
        except Exception:
            time.sleep(0.3)
    return False

def run_demo():
    print('Seeding database...')
    create_db()

    print('Starting MCP server...')
    proc = start_mcp_server()
    try:
        ok = wait_for_server()
        if not ok:
            print('MCP server did not start in time.')
            proc.terminate()
            return

        # Import agents after server is up to avoid requests race
        from agents import CustomerDataAgent, SupportAgent, RouterAgent

        print('Creating agents...')
        data_agent = CustomerDataAgent('DataAgent')
        support_agent = SupportAgent('SupportAgent', data_agent)
        router = RouterAgent('Router', data_agent, support_agent)

        scenarios = [
            ("Simple Query", "Get customer information for ID 5"),
            ("Coordinated Query", "I'm customer 12345 and need help upgrading my account"),
            ("Complex Query", "Show me all active customers who have open tickets"),
            ("Escalation", "I've been charged twice, please refund immediately!"),
            ("Multi-Intent", "Update my email to new@email.com and show my ticket history for 12345"),
        ]

        for name, q in scenarios:
            print('\n---')
            print(f"Scenario: {name}")
            print(f"Query: {q}")
            if 'update my email' in q.lower():
                # call support via router but include new_email in context
                resp = router.route(q + ' new@email.com')
            else:
                resp = router.route(q)
            print('Response:')
            print(resp)

    finally:
        print('Stopping MCP server...')
        proc.terminate()

# Run the demo when executing this cell
run_demo()

print('\n=== Demo Complete ===')
print('The notebook demonstrates:')
print('1. Task Allocation: Router -> DataAgent fetches customer data via MCP')
print('2. Multi-Step Coordination: Router decomposes complex queries into sub-tasks')
print('3. Explicit A2A Logging: Each agent handoff is logged showing coordination flow')

Seeding database...
Starting MCP server...
Creating agents...

---
Scenario: Simple Query
Query: Get customer information for ID 5
[Router] Routing query: Get customer information for ID 5
[Router] Detected intents: ['get_customer']
[Router] A2A: Router -> DataAgent: fetch customer 5
[DataAgent] Fetching customer 5 via MCP
[Router] A2A: DataAgent -> Router: returned customer data
Response:
{'reply': "Customer 5: {'created_at': '2025-11-25T21:08:50.132759', 'email': 'eve@example.com', 'id': 5, 'name': 'Eve Turner', 'phone': '555-0105', 'status': 'active', 'updated_at': '2025-11-25T21:08:50.132759'}"}

---
Scenario: Coordinated Query
Query: I'm customer 12345 and need help upgrading my account
[Router] Routing query: I'm customer 12345 and need help upgrading my account
[Router] Detected intents: ['support']
[Router] A2A: Router -> SupportAgent: forward to support
[SupportAgent] Handling support with context: {'text': "I'm customer 12345 and need help upgrading my account", 'customer_id'